<a href="https://colab.research.google.com/github/kalleo1996/pysparkMLExample/blob/main/Copy_of_Copy_of_Copy_of_Kalpani_predict_randomforestclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 64kB/s 
     |████████████████████████████████| 204kB 15.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=67a8208d0abb807adec12f8dcdf116e3846a4059c2107f286a970d13a5305e05
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession \
    .builder \
    .config("spark.driver.memory", "15g") \
    .appName("Exploratory Analysis") \
    .getOrCreate()

In [7]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [5]:
parking2017 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/content/gdrive/My Drive/Big Data/Parking_Violations_Issued_-_Fiscal_Year_2017.csv')
parking2016 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/content/gdrive/My Drive/Big Data/Parking_Violations_Issued_-_Fiscal_Year_2016.csv')
parking2015 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/content/gdrive/My Drive/Big Data/Parking_Violations_Issued_-_Fiscal_Year_2015.csv')
parking2014 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/content/gdrive/My Drive/Big Data/Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_.csv')

In [6]:
parking2017 = parking2017.dropDuplicates()
parking2016 = parking2016.dropDuplicates()
parking2015 = parking2015.dropDuplicates()
parking2014 = parking2014.dropDuplicates()

In [7]:
parkingtable2017 = parking2017.toDF(*(c.replace(' ', '_') for c in parking2017.columns))
parkingtable2016 = parking2016.toDF(*(c.replace(' ', '_') for c in parking2016.columns))
parkingtable2015 = parking2015.toDF(*(c.replace(' ', '_') for c in parking2015.columns))
parkingtable2014 = parking2014.toDF(*(c.replace(' ', '_') for c in parking2014.columns))

In [8]:
parkingtable2017.createOrReplaceTempView("parking2017")
parkingtable2016.createOrReplaceTempView("parking2016")
parkingtable2015.createOrReplaceTempView("parking2015")
parkingtable2014.createOrReplaceTempView("parking2014")

In [9]:
parkingtable2017 = spark.sql("select Issue_Date,Violation_Location, Violation_Code from parking2017")
parkingtable2016 = spark.sql("select Issue_Date,Violation_Location, Violation_Code from parking2016")
parkingtable2015 = spark.sql("select Issue_Date,Violation_Location, Violation_Code from parking2015")
parkingtable2014 = spark.sql("select Issue_Date,Violation_Location, Violation_Code from parking2014")

# Predictions

In [10]:
from pyspark.sql.functions import unix_timestamp, from_unixtime

combinedparkingtable = parkingtable2017.union(parkingtable2016).union(parkingtable2015).union(parkingtable2014)
combinedparkingtable.show()

+----------+------------------+--------------+
|Issue_Date|Violation_Location|Violation_Code|
+----------+------------------+--------------+
|02/22/2017|                19|            18|
|01/27/2017|               110|            74|
|07/23/2016|                71|            20|
|06/23/2017|                30|            21|
|06/15/2017|              null|            36|
|02/16/2017|              null|            36|
|10/28/2016|                72|            21|
|05/30/2017|                90|            21|
|08/20/2016|                47|            16|
|11/11/2016|                 9|            40|
|02/16/2017|                40|            21|
|12/04/2016|                32|            71|
|03/09/2017|              null|             7|
|10/13/2016|              null|             5|
|04/10/2017|                24|            21|
|10/04/2016|                23|            38|
|05/08/2017|               110|            13|
|10/29/2016|               112|            37|
|03/15/2017| 

In [11]:
#parking2017.show(100)

In [12]:
combinedparkingtable=combinedparkingtable.dropna()

In [13]:
combinedparkingtable.count()


35236230

In [14]:
combinedparkingtable.createOrReplaceTempView("combinedparkingtableView")


In [15]:
def get_weekday(date):
    import datetime
    import calendar
    month, day, year = (int(x) for x in date.split('/'))    
    weekday = datetime.date(year, month, day)
    #return calendar.day_name[weekday.weekday()]
    
    if calendar.day_name[weekday.weekday()] == 'Sunday':
      return 0
    if calendar.day_name[weekday.weekday()] == 'Monday':
      return 1
    if calendar.day_name[weekday.weekday()] == 'Tuesday':
      return 2
    if calendar.day_name[weekday.weekday()] == 'Wednesday':
      return 3
    if calendar.day_name[weekday.weekday()] == 'Thursday':
      return 4
    if calendar.day_name[weekday.weekday()] == 'Friday':
      return 5
    if calendar.day_name[weekday.weekday()] == 'Saturday':
      return 6
    if calendar.day_name[weekday.weekday()] == 'Sunday':
      return 7

spark.udf.register('get_weekday', get_weekday)

<function __main__.get_weekday>

In [16]:
from pyspark.sql.functions import udf

def week_number(date_str):
    start_day_of_week=7
   
    def _week_number(date_str):
        from datetime import datetime, date
        d = datetime.strptime(date_str, '%m/%d/%Y')     # compatible with Python2
        wd_d1 = date(d.year, d.month, 1).isoweekday()
        offset = (wd_d1 - start_day_of_week + 7 ) % 7
        return (d.day - 1 + offset) // 7 + 1
    return _week_number(date_str)
spark.udf.register('week_number',week_number)    

<function __main__.week_number>

In [17]:
from pyspark.sql.functions import udf,desc
from datetime import datetime

In [18]:


from pyspark.sql.functions import date_format
from pyspark.sql.functions import to_date, date_format

from pyspark.sql.functions import *


for 2014 to 2016 first

In [19]:
data_2014 = spark.sql("select year(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as year,month(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as month,week_number(Issue_Date) as week,get_weekday(Issue_Date) as day,Violation_Location, Violation_Code , count(*) as frequency from combinedparkingtableView where   Group by year,month,week,day,Violation_Location, Violation_Code order by year,month,week,day,Violation_Location,Violation_Code ")
data_2014.createOrReplaceTempView("data_2014")
data_2014 = spark.sql(" SELECT CAST(year AS INTEGER),CAST(month AS INTEGER),CAST(week AS INTEGER) , CAST(day AS INTEGER), CAST(Violation_Location AS INTEGER),CAST(Violation_Code AS INTEGER),frequency FROM data_2014 where year>= 2014 ")




In [20]:
data_2014.dtypes

[('year', 'int'),
 ('month', 'int'),
 ('week', 'int'),
 ('day', 'int'),
 ('Violation_Location', 'int'),
 ('Violation_Code', 'int'),
 ('frequency', 'bigint')]

In [21]:
data_2014_day = spark.sql("select year(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as year,month(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as month,week_number(Issue_Date) as week,get_weekday(Issue_Date) as day,Violation_Location,count(*) as totalViolation from combinedparkingtableView  Group by year,month,week,day,Violation_Location order by year,month,week,day,Violation_Location ")
data_2014_day.createOrReplaceTempView("data_2014_d")
data_2014_day=spark.sql(" SELECT CAST(year AS INTEGER),CAST(month AS INTEGER),CAST(week AS INTEGER) , CAST(day AS INTEGER), CAST(Violation_Location AS INTEGER),totalViolation FROM data_2014_d where year >= 2014")



In [22]:
data_2014_day.dtypes

[('year', 'int'),
 ('month', 'int'),
 ('week', 'int'),
 ('day', 'int'),
 ('Violation_Location', 'int'),
 ('totalViolation', 'bigint')]

In [23]:
import pyspark.sql.functions as F
from pyspark.sql.functions import round, col
df_2014 = data_2014.join(data_2014_day, (data_2014.Violation_Location == data_2014_day.Violation_Location) & (data_2014.year == data_2014_day.year)& (data_2014.month == data_2014_day.month)& (data_2014.week == data_2014_day.week) & (data_2014_day.day == data_2014_day.day))\
    .withColumn("violation_probability", round(F.col("frequency") / F.col("totalViolation"), 5))\
    .drop(data_2014_day.totalViolation).drop(data_2014.frequency).drop(data_2014_day.Violation_Location).drop(data_2014_day.month).drop(data_2014_day.week).drop(data_2014_day.day).drop(data_2014_day.year)



In [24]:
#dataframe concatenation
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

#def unionAll(*dfs):
    #return reduce(DataFrame.unionAll, dfs)

#full_df = unionAll( df_2015, df_2016,df)
#train_df.show(10)

In [25]:
full_df = df_2014


In [26]:
#from pyspark.ml.feature import StringIndexer
#qualification_indexer = StringIndexer(inputCol="day", outputCol="day_chng")
#Fits a model to the input dataset with optional parameters.
#df_buck = qualification_indexer.fit(full_df).transform(full_df)
#df_buck.show()

In [27]:
from pyspark.ml.feature import OneHotEncoder

#for training data
#

#day_encoded = OneHotEncoder(inputCol="day", outputCol="day_one")
#df_buck = day_encoded.fit(full_df).transform(full_df)

#week_encoded = OneHotEncoder(inputCol="week", outputCol="week_one")
#df_buck = week_encoded.fit(df_buck).transform(df_buck)

month_encoded = OneHotEncoder(inputCol="month", outputCol="month_one")
df_buck = month_encoded.fit(full_df).transform(full_df)

location_encoded = OneHotEncoder(inputCol="Violation_Location", outputCol="Violation_Location_one")
df_buck1 = location_encoded.fit(df_buck).transform(df_buck)

code_encoded = OneHotEncoder(inputCol="Violation_Code", outputCol="Violation_Code_one")
df_buck2 = code_encoded.fit(df_buck1).transform(df_buck1)


#df_buck_train.show(10)

In [28]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

#for training data
#assembler = VectorAssembler(inputCols=["day_one","month_one","Violation_Location_one","week_one","Violation_Code_one"], outputCol="features")
# Assemble all the features with VectorAssembler
required_features = ["year","month_one","week","day","Violation_Location_one","Violation_Code_one"]

from pyspark.ml.feature import VectorAssembler

numericCols = [] #['prob']
assemblerInputs = [c  for c in required_features] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol='features')

transformed_data = assembler.transform(df_buck2)
#transformed_data.show(5)

In [29]:
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2], seed =2020)

In [30]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='violation_probability')
lr_model = lr.fit(training_data)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))


Coefficients: [0.010212225045646357,0.0,0.07257204241195582,-0.036191404321141075,-0.0368918811894584,-0.058208596212540875,-0.04931796961802898,-0.03556383565506816,-0.03928115587930469,-0.056099835807713994,-0.0402245660668567,-0.05338902437132868,-0.03482070016621827,0.011343466258329712,0.0004998534110976042,0.0,0.021763218026816605,0.6336216101850222,0.7779506918617642,0.6844795756153135,-0.029071890870521904,0.04434859036889491,-0.03433911131440282,0.7088880651990085,0.016899534147072546,-0.020539907982423174,0.7320827320244391,0.7177346634731776,-0.006825079019406478,-0.019482049388238252,0.7240402491079476,0.71372408791793,-0.005610623752594327,-0.013532291036015736,0.005287365393718115,-0.012661980409103152,0.7385319080831111,0.7551944967509239,-0.020343509036775776,-0.0052579980971528256,-0.02228126981445335,0.020096253844987748,1.3439558517534336,-0.022732441252720604,0.789949566662772,-0.018824354182431353,0.803104913159033,-0.0325984316957795,-0.017407882011311403,-0.02701

In [31]:
model_path = "/content/gdrive/My Drive/Big Data/" + "modeldaynew"
lr_model.save(model_path)

In [1]:
model_path = "/content/gdrive/My Drive/Big Data/" + "modeldaynew"

In [8]:
from pyspark.ml.regression import LinearRegressionModel
model2 = LinearRegressionModel.load(model_path)

In [33]:
model2_predict = model2.transform(test_data.limit(10))

In [34]:
model2_predict.select("prediction","violation_probability","features").show(10)

+-------------------+---------------------+--------------------+
|         prediction|violation_probability|            features|
+-------------------+---------------------+--------------------+
| 0.1731436067019061|              0.05714|(1108,[0,2,13,14,...|
|0.19214876003147552|              0.05714|(1108,[0,2,13,14,...|
|0.07188895032748377|              0.01429|(1108,[0,2,13,14,...|
|0.07188895032748377|              0.07692|(1108,[0,2,13,14,...|
|0.12197789145616511|              0.01429|(1108,[0,2,13,14,...|
|0.08391810212247464|              0.11765|(1108,[0,2,13,14,...|
| 0.0796079068911304|              0.07692|(1108,[0,2,13,14,...|
|0.07440072002006204|              0.04545|(1108,[0,2,13,14,...|
|0.22332858449577486|              0.23134|(1108,[0,2,13,14,...|
|  0.098063552995864|              0.03226|(1108,[0,2,13,14,...|
+-------------------+---------------------+--------------------+



In [35]:

print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.010212225045646357,0.0,0.07257204241195582,-0.036191404321141075,-0.0368918811894584,-0.058208596212540875,-0.04931796961802898,-0.03556383565506816,-0.03928115587930469,-0.056099835807713994,-0.0402245660668567,-0.05338902437132868,-0.03482070016621827,0.011343466258329712,0.0004998534110976042,0.0,0.021763218026816605,0.6336216101850222,0.7779506918617642,0.6844795756153135,-0.029071890870521904,0.04434859036889491,-0.03433911131440282,0.7088880651990085,0.016899534147072546,-0.020539907982423174,0.7320827320244391,0.7177346634731776,-0.006825079019406478,-0.019482049388238252,0.7240402491079476,0.71372408791793,-0.005610623752594327,-0.013532291036015736,0.005287365393718115,-0.012661980409103152,0.7385319080831111,0.7551944967509239,-0.020343509036775776,-0.0052579980971528256,-0.02228126981445335,0.020096253844987748,1.3439558517534336,-0.022732441252720604,0.789949566662772,-0.018824354182431353,0.803104913159033,-0.0325984316957795,-0.017407882011311403,-0.02701

In [36]:
lr_predictions = lr_model.transform(test_data)
lr_predictions.select("prediction","violation_probability","features").show(100)

+--------------------+---------------------+--------------------+
|          prediction|violation_probability|            features|
+--------------------+---------------------+--------------------+
|  0.1731436067019061|              0.05714|(1108,[0,2,13,14,...|
| 0.19214876003147552|              0.05714|(1108,[0,2,13,14,...|
| 0.07188895032748377|              0.01429|(1108,[0,2,13,14,...|
| 0.07188895032748377|              0.07692|(1108,[0,2,13,14,...|
| 0.12197789145616511|              0.01429|(1108,[0,2,13,14,...|
| 0.08391810212247464|              0.11765|(1108,[0,2,13,14,...|
|  0.0796079068911304|              0.07692|(1108,[0,2,13,14,...|
| 0.07440072002006204|              0.04545|(1108,[0,2,13,14,...|
| 0.22332858449577486|              0.23134|(1108,[0,2,13,14,...|
|   0.098063552995864|              0.03226|(1108,[0,2,13,14,...|
| 0.08897499132313413|              0.01493|(1108,[0,2,13,14,...|
| 0.08897499132313413|              0.03509|(1108,[0,2,13,14,...|
| 0.228413

In [37]:
dataForGrapgh = lr_predictions.select("prediction","violation_probability","features")

In [ ]:
dataG =dataForGrapgh.toPandas()

In [ ]:
# Necessary imports: 
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
accuracy = metrics.r2_score(dataG['violation_probability'], dataG['prediction'])
print("Cross-Predicted Accuracy:", accuracy)

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (24, 8)
plt.plot(dataG['prediction'][:100], label = "Pred")  # Load the 500 data points from prediction with label name 'Pred'
plt.plot(dataG['violation_probability'][:100], label = "Actual")